### All Code Comparison

In [1]:
import os
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from utils import *
from hamming_channel_47 import HammingChannel47
from ldpc_channel import LdpcChannel
import matplotlib.pyplot as plt
from ldpc_bpsk_gaussian_system import LdpcBpskGaussianSystem

#### Hamming

In [2]:
# 1. Escolha um valor de p. Utilize p = 0.5,0.2,0.1,0.05,0.02,0.01,0.005,...

snrs_db = np.arange(0, 5.5, 0.5) 
ps = []
R = 4/7
for snr_db in snrs_db:
    ps.append(Ei_N0_to_p(snr_db, R)) 
    
bit_values = [0, 1]
save_folder = "./bit_error_results"
if not os.path.exists(save_folder):
    os.mkdir(save_folder)
method = "hamming"
for p in ps:
    print(f"p = {p}")
    
    # 2. Gere aleatoriamente cerca de 1 milhão de bits de informação
    # 3. Divida em grupos de K bits. Haverá L grupos
    save_path_original = save_folder + f"/{p}-{method}-bits-original.npy"    

    if not os.path.isfile(save_path_original):  
        random_info_bits = np.array(random.choices(bit_values, weights=[1, 1], k = int(1e6))) 
        k = 4
        original_bits_groups = random_info_bits.reshape(-1, k) 
        with open(save_path_original, "wb") as f:
            np.save(f, original_bits_groups)
    else:
        with open(save_path_original, "rb") as f:
            original_bits_groups = np.load(f)
            
    # 4: Gere as L palavras-código correspondentes a cada um dos grupos do item 3
    # 5: Simule o efeito do canal BSC trocando o valor dos bits das palavras código do item anterior 
    # com probabilidade p para todos os bits de todas as L palavras-código, gerando assim L palavras recebidas.
    # 6: Realize o processo de detecção por síndrome para cada uma das L palavras recebidas.
    # 7: Realize o processo de estimação sobre os bits de informação.
    save_path_decoded = save_folder + f"/{p}-{method}-bits-decoded.npy"

    if not os.path.isfile(save_path_decoded):
        hamming_channel = HammingChannel47(p)
        decoded_bits_groups = []
        for i in tqdm(range(original_bits_groups.shape[0])):
            decoded_bits_groups.append(hamming_channel.transmit(original_bits_groups[i])) 
        decoded_bits_groups = np.array(decoded_bits_groups)
        with open(save_path_decoded, 'wb') as f:
            np.save(f, decoded_bits_groups)
    else: 
        with open(save_path_decoded, "rb") as f:
            decoded_bits_groups = np.load(f)

p = 0.14252470370130643


100%|██████████| 250000/250000 [01:11<00:00, 3502.16it/s]


p = 0.12873528209043172


100%|██████████| 250000/250000 [01:01<00:00, 4047.44it/s]


p = 0.11516908862773423


...

100%|██████████| 250000/250000 [01:10<00:00, 3542.69it/s]


p = 0.03634940131838471


100%|██████████| 250000/250000 [01:05<00:00, 3825.36it/s]


p = 0.028647034206930377


100%|██████████| 250000/250000 [01:02<00:00, 4030.59it/s]


In [4]:
save_folder = "./bit_error_results"
info_bit_error_probas = []

for p in ps:
    save_path_original = save_folder + f"/{p}-{method}-bits-original.npy"
    with open(save_path_original, "rb") as f:
        original_bits_groups = np.load(f)

    save_path_decoded = save_folder + f"/{p}-{method}-bits-decoded.npy"
    with open(save_path_decoded, "rb") as f:
        decoded_bits_groups = np.load(f)
        
    original = original_bits_groups.reshape(-1)
    decoded = decoded_bits_groups.reshape(-1)
    info_bit_error_proba = np.not_equal(original, decoded).sum()/len(original)
    info_bit_error_probas.append(info_bit_error_proba)

In [6]:
hamming_errors = info_bit_error_probas

In [21]:
hamming_errors

[0.119709,
 0.100886,
 0.085127,
 0.069136,
 0.05474,
 0.042593,
 0.031849,
 0.023361,
 0.015978,
 0.010554,
 0.00705]

### LDPC - Bit Flipping

In [15]:
#4/7 Rate
rate="4/7"
save_folder = "./bit_error_results_47"
hamming=False
dv=3
dc=7
N_values=[1001]
snrs_db = np.arange(0, 5.5, 0.5) 
ps = []
R = 4/7
for snr_db in snrs_db:
    ps.append(Ei_N0_to_p(snr_db, R)) 

In [16]:
def generate_groups(N, N_bits, ps):
    """
    dv = 3 # número de ramos que saem de cada v-node para a camada CND = número de 1s nas colunas
    dc = 7 # número de ramos que saem de cada c-node para a camada VND = número de 1s nas linhas
    N = 98 # tamanho da palavra código
    """
    n_groups = int(N_bits/N)
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    method = "ldpc"

    for p in ps:
        save_path_original = save_folder + f"/{N}-{p}-{method}-bits-original.npy"    

        if not os.path.isfile(save_path_original):  
            original_bits_groups = np.zeros((1, N_bits)).reshape(-1, N) 
            with open(save_path_original, "wb") as f:
                np.save(f, original_bits_groups)

        save_path_decoded = save_folder + f"/{N}-{p}-{method}-bits-decoded.npy"
        if not os.path.isfile(save_path_decoded):
            print(f"p = {p}")
            ldpc_channel = LdpcChannel(p, dv, dc, N, iterations=100)
            decoded_bits_groups = []
            for i in tqdm(range(n_groups)):
                decoded_bits_groups.append(ldpc_channel.transmit()) 
            decoded_bits_groups = np.array(decoded_bits_groups)
            with open(save_path_decoded, 'wb') as f:
                np.save(f, decoded_bits_groups)
        else: 
            with open(save_path_decoded, "rb") as f:
                decoded_bits_groups = np.load(f)

def get_info_bit_error_probas(N, ps):
    method = "ldpc"
    info_bit_error_probas = []

    for p in ps:
        save_path_original = save_folder + f"/{N}-{p}-{method}-bits-original.npy"
        with open(save_path_original, "rb") as f:
            original_bits_groups = np.load(f)

        save_path_decoded = save_folder + f"/{N}-{p}-{method}-bits-decoded.npy"
        with open(save_path_decoded, "rb") as f:
            decoded_bits_groups = np.load(f)
            
        original = original_bits_groups.reshape(-1)
        decoded = decoded_bits_groups.reshape(-1)
        info_bit_error_proba = np.not_equal(original, decoded).sum()/len(original)
        info_bit_error_probas.append(info_bit_error_proba)
        
    return info_bit_error_probas

def get_probas_list_ldpc(N_values, ps):
    
    if hamming:
        info_bit_error_probas_hamming = []
        method = "hamming"

        for p in ps:
            save_path_original = save_folder + f"/{p}-{method}-bits-original.npy"
            with open(save_path_original, "rb") as f:
                original_bits_groups = np.load(f)

            save_path_decoded = save_folder + f"/{p}-{method}-bits-decoded.npy"
            with open(save_path_decoded, "rb") as f:
                decoded_bits_groups = np.load(f)
                
            original = original_bits_groups.reshape(-1)
            decoded = decoded_bits_groups.reshape(-1)
            info_bit_error_proba = np.not_equal(original, decoded).sum()/len(original)
            info_bit_error_probas_hamming.append(info_bit_error_proba)
    
    probas_list = []
    
    for N in N_values:
        generate_groups(N=N, N_bits=N*1000, ps=ps)
        info_bit_error_probas = get_info_bit_error_probas(N=N, ps=ps)
        for i in range(len(info_bit_error_probas) - 1):
            if(info_bit_error_probas[i] == 0 and info_bit_error_probas[i+1] > 0):
                info_bit_error_probas[i+1] = 0
        probas_list.append(info_bit_error_probas)
    
    return probas_list

In [17]:
ldpc_errors = get_probas_list_ldpc(N_values, ps)

p = 0.14252470370130643


100%|██████████| 1000/1000 [04:26<00:00,  3.75it/s]


p = 0.12873528209043172


100%|██████████| 1000/1000 [04:42<00:00,  3.54it/s]


p = 0.11516908862773423


...

100%|██████████| 1000/1000 [03:51<00:00,  4.32it/s]


p = 0.03634940131838471


100%|██████████| 1000/1000 [03:20<00:00,  4.99it/s]


p = 0.028647034206930377


100%|██████████| 1000/1000 [02:56<00:00,  5.67it/s]


In [20]:
ldpc_errors = ldpc_errors[0]
ldpc_errors

[0.19182517482517483,
 0.17631668331668332,
 0.15798201798201797,
 0.14072227772227772,
 0.1195964035964036,
 0.09866333666333667,
 0.0764115884115884,
 0.05357142857142857,
 0.03307992007992008,
 0.016977022977022978,
 0.007014985014985015]

### LDPC - Belief Propagation

In [2]:
N = 1001
n_groups = 100
N_bits = N * n_groups
dv = 3
dc = 7
max_iter = 50
Eb = 1
R = 4/7
snr_db_values_Ei = np.arange(0, 5.5, 0.5)
snr_db_values_Eb = [Ei_snr_db_to_Eb_snr_db(i, R) for i in snr_db_values_Ei]
N0_values = [N0_from_db_value(i, Eb) for i in snr_db_values_Eb]
original = np.zeros(N_bits)
info_bit_error_probas = []
uncoded_info_bit_error_probas = []

for N0 in N0_values:
    system = LdpcBpskGaussianSystem(N0, Eb, dv, dc, N, max_iter)
    decoded_bits_groups = []
    uncoded_bits_groups = []
    for i in tqdm(range(n_groups)):
        decoded_bits_groups.append(system.apply_noise_and_decode()) 
        uncoded_bits_groups.append(system.get_uncoded_bits())
    decoded_bits_groups = np.array(decoded_bits_groups)
    uncoded_bits_groups = np.array(uncoded_bits_groups)
    decoded = decoded_bits_groups.reshape(-1)
    uncoded = uncoded_bits_groups.reshape(-1)
    info_bit_error_proba = np.not_equal(original, decoded).sum()/len(original)
    info_bit_error_probas.append(info_bit_error_proba)
    uncoded_info_bit_error_proba = np.not_equal(original, uncoded).sum()/len(original)
    uncoded_info_bit_error_probas.append(uncoded_info_bit_error_proba)
    
ldpc_belief_errors = info_bit_error_probas

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:21<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
ldpc_belief_errors